In [1]:
#!pip install pandas
#!pip install openpyxl
#!pip install epitran
#!pip install pronouncing
#!pip install rhymetagger


In [2]:
import pandas as pd
import string
import re
import os
from rhymetagger import RhymeTagger

In [3]:
df = pd.read_csv("паспорт хорошего русского - песня скомпилированная рифма AABB.csv",)

In [4]:
path_to_file = "kino-kukushka.txt"
path_to_file = "glinka-slavsa.txt"
path_to_file = "lube-kon.txt"

path_to_file = "bernes-temnaya-noc.txt"
path_to_file = "tscvetaeva.txt"

In [5]:
with open(path_to_file) as f:
    text_song = f.readlines()
#text_song

df = pd.DataFrame(text_song, columns = ["text"])
df = df[df["text"] != "\n"]
df = df.reset_index(drop = True)

l1 = []
df2 = pd.DataFrame(columns = ["text"])
for ind in df.index:
    n1 = ind + 1
    n = n1 % 4
    if n == 1: 
        l1 = [df['text'][ind]]
    else:
        l1.append(df['text'][ind])
    if n == 0: 
        text = "".join(l1)
        new_row = pd.DataFrame([text], columns=['text'])
        df2 = pd.concat([df2, new_row], ignore_index=True)
        #text = ""

df2["text"] = df2.apply(lambda row: row["text"][:-1] if row["text"][-1] else row["text"], axis = 1)
df = df2

#df2.values

In [6]:
def get_item(text):
    arr = text.split("\n")
    #l1 = list(map(lambda x: x if x[-1].isalpha() else x[:-1], arr))
    
    def remove_punctuation(str):
        #str = str if str[-1].isalpha() else str[:-1]
        #return str.translate(str.maketrans('', '', string.punctuation))
        return re.sub(r'[^\w\s]', '', str).strip()

    arr = list(map(remove_punctuation,arr))
    return  arr
    

In [7]:
#get_item(df.iloc[0].values[0])
df["text_splitted"] = df.apply(lambda raw:get_item(raw["text"]), axis=1)

In [8]:
#df.iloc[5]["text"], df.iloc[5]["text_splitted"], 

In [9]:
def get_rhyme_score(text1, text2):

    #This function counts only the direct equals of symbols from the end of phrases 
    #and isn`t aware of fonetical equals
    
    n = 1
    while (n < len(text1) or n < len(text2)):
        c1 = text1[-n]
        c2 = text2[-n] 
        #print(c1, c2)
        if c1 == c2:
            n = n+1
        else:
            break
    return n - 1

In [10]:
"A" , get_rhyme_score(df.iloc[0]["text_splitted"][0], df.iloc[0]["text_splitted"][2])
#, get_rhyme_score(df[0][0], df[0][2]), get_rhyme_score(df[0][0], df[0][3])

('A', 2)

In [11]:
def get_rhyme_score_of_quatrain_old(l4_quatrain):
    l1 = l4_quatrain
    letters = "ABCD"
    n = len(letters)
    scores = [None] * n 
    for i in range(0,n):
        current_letter = letters[i]
        if scores[i] is None: 
#            scores[i] = {current_letter:100}
            scores[i] = {current_letter}
        for j in range(i+1,n):
            score = get_rhyme_score(l1[i], l1[j])
            if (score > 0):
                scores[j] = {current_letter:score}
    return scores

In [12]:
rt = RhymeTagger()
rt.load_model(model='ru')

Model loaded with following settings:
  frequency_min: 3
           lang: ru
       max_iter: 20
          ngram: 3
   ngram_length: 3
   prob_ipa_min: 0.9
 prob_ngram_min: 0.9
     same_words: False
   stanza_limit: True
         stress: True
       syll_max: 2
    t_score_min: 3.078
   vowel_length: True
         window: 5


In [13]:
def get_rhyme_score_of_quatrain(l4_quatrain):
    poem = l4_quatrain
    #rt.add_to_model(poem) 
    rhymes = rt.tag(poem, output_format=3) 
    return rhymes


In [24]:
def get_rhyme_score_of_quatrain_literal_and_fonetical(l4_quatrain):
    l1 = get_rhyme_score_of_quatrain(l4_quatrain)
    if None in l1:
        l2 = get_rhyme_score_of_quatrain_old(l4_quatrain)
    for i in range(len(l1)):
        if l1[i] is None:
            l1[i] = l2[i]
    return l1

In [25]:
l1 = df.iloc[0]["text_splitted"]
l1 , get_rhyme_score_of_quatrain_literal_and_fonetical(l1)

(['Уж сколько их упало в эту бездну',
  'Разверстую вдали',
  'Настанет день когда и я исчезну',
  'С поверхности земли'],
 [1, 2, 1, 2])

In [26]:
df["scores"] = df.apply(lambda raw: get_rhyme_score_of_quatrain_literal_and_fonetical(raw["text_splitted"]), axis = 1)

In [27]:
df

,text,text_splitted,scores
0,"Уж сколько их упало в эту бездну,\nРазверстую ...","[Уж сколько их упало в эту бездну, Разверстую ...","[1, 2, 1, 2]"
1,"Застынет всё, что пело и боролось,\nСияло и рв...","[Застынет всё что пело и боролось, Сияло и рва...","[1, 1, 2, 2]"
2,"И будет жизнь с её насущным хлебом,\nС забывчи...","[И будет жизнь с её насущным хлебом, С забывчи...","[1, 2, 1, 2]"
3,"Изменчивой, как дети, в каждой мине,\nИ так не...","[Изменчивой как дети в каждой мине, И так недо...","[1, 2, 1, 2]"
4,"Виолончель и кавалькады в чаще,\nИ колокол в с...","[Виолончель и кавалькады в чаще, И колокол в с...","[{A}, 1, {C}, 1]"
5,"— К вам всем — что мне, ни в чём не знавшей ме...","[К вам всем что мне ни в чём не знавшей меры,...","[1, 2, 1, 2]"
6,"И день и ночь, и письменно и устно:\nЗа правду...","[И день и ночь и письменно и устно, За правду ...","[1, 2, 1, 2]"
7,"За то, что мне — прямая неизбежность —\nПрощен...","[За то что мне прямая неизбежность, Прощение ...","[1, 2, 1, 2]"
8,"За быстроту стремительных событий,\nЗа правду,...","[За быстроту стремительных событий, За правду ...","[{A}, 1, {C}, 1]"


In [17]:
file_name = f"{os.path.splitext(path_to_file)[0]}.xlsx"
#file_name
df.loc[:, ["text","scores"]].to_excel(file_name)

### Let`s control the different cases:

In [18]:
df.iloc[0].values
#here is Ok, ABAB

array(['Уж сколько их упало в эту бездну,\nРазверстую вдали!\nНастанет день, когда и я исчезну\nС поверхности земли.',
       list(['Уж сколько их упало в эту бездну', 'Разверстую вдали', 'Настанет день когда и я исчезну', 'С поверхности земли']),
       list([1, 2, 1, 2])], dtype=object)

In [19]:
df.iloc[1].values[1]
#here is ABAC, should be AABB
#because phonetically ах ~ аш

['Застынет всё что пело и боролось',
 'Сияло и рвалось',
 'И зелень глаз моих и нежный голос',
 'И золото волос']

In [20]:
#df.iloc[6].values
#here is ABCC, should be AABB 
#because phonetically ных ~ ну

### Conclusion: the get_rhyme_score(text1, text2) should be extended for fonetical equal phrases
## theory here https://stihi.ru/2011/10/19/7548#:~:text=%D0%9D%D0%B5%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F%20%D1%80%D0%B8%D1%84%D0%BC%D0%B0%20%D1%85%D0%B0%D1%80%D0%B0%D0%BA%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D1%83%D0%B5%D1%82%D1%81%D1%8F%20%D1%80%D0%B0%D0%B7%D0%BB%D0%B8%D1%87%D0%B8%D1%8F%D0%BC%D0%B8%20%D0%B2,%D1%81%D0%BB%D0%BE%D0%B3%D0%B0%D1%85%2C%20%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B2%20%D0%BE%D0%BA%D0%BE%D0%BD%D1%87%D0%B0%D0%BD%D0%B8%D1%8F%D1%85%20%D1%81%D1%82%D1%80%D0%BE%D0%BA.


In [21]:
def is_Consonance():

    #https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%81%D0%BE%D0%BD%D0%B0%D0%BD%D1%81_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F)
    return False

def is_Assonance():

    #https://en.wikipedia.org/wiki/Assonance
    return False
    


In [22]:
from epitran.backoff import Backoff

backoff = Backoff(['rus-Cyrl', 'rus-Cyrl'])
                  
backoff.transliterate('За быстроту стремительных событий,За правду, за игру…— Послушайте! — Ещё меня любите За то, что я умру')

'za bɨstrotu strʲemʲitʲelʲnɨx sobɨtʲiй,za pravdu, za iɡru…— posluʂaйtʲe! — jeɕːё mʲena lʲubʲitʲe za to, t͡ɕʲto ja umru'

In [23]:
import pronouncing
pronouncing.rhymes("стремительных")


[]